In [ ]:
import numpy as np
import gym

In [ ]:
from tiles3 import tiles, IHT  # This is the package for constructing the feature tilings.

In [ ]:
def selector(actions, values, epsilon):
    greedy = np.random.choice([1,0], p=[1-epsilon, epsilon])
    if greedy:
        return actions[np.argmax(values)]
    else:
        return np.random.choice(actions)

In [ ]:
n_episodes = 100
n_timesteps = 1500
n_tilings = 10
tiling_dim = 10
max_tile_size = 800 # ?? how to pick
iht = IHT(max_tile_size)

In [ ]:
env = gym.make('MountainCar-v0')
env._max_episode_steps = n_timesteps
n_action = env.action_space.n
obs_highs = env.observation_space.high
obs_lows = env.observation_space.low
scales = [tiling_dim / (obs_highs[i] - obs_lows[i]) for i in range(len(obs_highs))]

# theta = np.zeros(max_tile_size + env.action_space.n)  # last n spots indicate which action
theta = np.zeros(max_tile_size *  3)

# Tiling helper function
# Because the tiles code only divides at integers, we must scale our observations to tiling_dim x tiling_dim space
def mytiles(pt, a):
    pt_ = np.array([p*scales[i] for i,p in enumerate(list(pt))])
    features = list(np.array(tiles(iht, n_tilings, pt_)) + a * max_tile_size)
    phi = np.zeros_like(theta)
    phi[features] = 1
    return phi

# init weights


In [ ]:
# params
epsilon = .5 # (greedy)
gamma = .9
lam = .9
alpha = 1. / (10 * n_tilings) # alpha is often 1/10n where n is number of tilings
actions = range(env.action_space.n)

for i_episode in range(n_episodes):
    epsilon *= .95
    epsilon = max(epsilon, .1)
    observation = env.reset()
    
    
#     phi_sa[-(n_action - action)] = 1  # turn on action feature
    phis = [mytiles(observation, a) for a in actions] # get features for observation
    q_vals = [theta.dot(phi) for phi in phis]
    action = selector(actions, q_vals, epsilon)
    phi_sa = phis[action]
    q_sa = q_vals[action]

    # initialize e = 0
    e = np.zeros_like(theta)
    
    for t in range(n_timesteps):
        env.render()
        
        # Take action, observe change, choose new a, get new features and q_sa_ and do update
        
        observation, reward, done, info = env.step(action)
        
        phis = [mytiles(observation, a) for a in actions]
        q_vals = [theta.dot(phi) for phi in phis]
        action = selector(actions, q_vals, epsilon)
        phi_sa_ = phis[action]
        q_sa_ = q_vals[action]
        
        
        d = reward + gamma * q_sa_ - q_sa
        e = gamma * lam * e + alpha * (1 - gamma * lam * e.dot(phi_sa)) * phi_sa
        theta = theta + d * e + alpha * (q_sa - theta.dot(phi_sa)) * phi_sa
        
        
        q_sa = q_sa_
        phi_sa = phi_sa_
    
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
    
env.close()

In [ ]:
theta[-3:]

In [ ]:
actions = [0,1,2]
values = [10, 20, 30]

In [ ]:
selector(actions, values, .1)

In [ ]:
# three separate theta vectors ? one per action? noooo
# include as nother dimension in tiling?
# one theta, three times as long, each phi *= action*max_state_size)